In [1]:
from sklearn import preprocessing , svm, model_selection
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import pandas as pd
import quandl, datetime
import math
import numpy as np
from sklearn import preprocessing 
import matplotlib.pyplot as plt
from matplotlib import style 
import pickle ##LINE ADDED
style.use('ggplot')

df = quandl.get('WIKI/GOOGL')
#print(df.head(1)) 

df = df.loc[:, ["Adj. Open", "Adj. High","Adj. Low","Adj. Close", "Adj. Volume"]]
#df.head()

df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.0
df['PCT_CHNG'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0
#df.head(3)

df = df[['Adj. Close', 'HL_PCT', 'PCT_CHNG', 'Adj. Volume']]
forecast_col = 'Adj. Close'
df.fillna(-99999, inplace = True)

forecast_out = int(math.ceil(0.01*len(df)))

df['label'] = df[forecast_col].shift(-forecast_out)


x = np.array(df.drop(['label'],1))
x= preprocessing.scale(x)
x=x[:-forecast_out]
x_lately = x[-forecast_out:]

df.dropna(inplace = True)
x = preprocessing.scale(x)
y = np.array(df['label'])


##print(len(x), len(y)) 
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size = 0.2)

clf = LinearRegression()
clf.fit(x_train, y_train)
with open('linearregression.pickle', 'wb') as f: 
   pickle.dump(clf,f)

pickle_in = open('linearregression.pickle', 'rb')
clf = pickle.load(pickle_in)

accuracy = clf.score(x_test,y_test)
#print(accuracy)
forecast_set = clf.predict(x_lately)

print(forecast_set, accuracy, forecast_out)

import matplotlib.pyplot as plt
from matplotlib import style 
style.use('ggplot')
import datetime

df['Forecast'] = np.nan
last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day

for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += one_day 
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)] + [i]

print(df.head())    
df['Adj. Close'].plot()
df['Forecast'].plot()
plt.legend(loc = 4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

[1048.22540886 1062.73366837 1076.31956948 1071.15753218 1064.81917099
 1062.26187324 1060.90024834 1058.00072152 1051.75560822 1047.46441611
 1045.43307484 1063.83204737 1081.58690818 1086.16495391 1100.73873388
 1104.58819918 1103.46743464 1101.02895354 1102.83166088 1120.26082697
 1119.00536897 1129.65473333 1126.19078749 1133.79229486 1152.87366889
 1165.27694249 1159.3164317  1171.36155667 1176.9883267  1174.36453596
 1165.46311877 1171.31416726 1169.73900509 1107.43464318 1054.03181779] 0.9808118672432518 35
            Adj. Close    HL_PCT  PCT_CHNG  Adj. Volume      label  Forecast
Date                                                                        
2004-08-19   50.322842  3.712563  0.324968   44659000.0  69.078238       NaN
2004-08-20   54.322689  0.710922  7.227007   22834300.0  67.839414       NaN
2004-08-23   54.869377  3.729433 -1.227880   18256100.0  68.912727       NaN
2004-08-24   52.597363  6.417469 -5.726357   15247300.0  70.668146       NaN
2004-08-25   53.16

<Figure size 640x480 with 1 Axes>